In [0]:
sc.addPyFile('https://raw.githubusercontent.com/microsoft/FabricBQSync/main/Packages/FabricSync/FabricSync/DeltaStorageInventory.py')

In [0]:
from DeltaStorageInventory import *

In [0]:
inventory_dt = None #Corresponds to the blob inventory snapshot, defaults to current day
inventory_rule = "Demo" #Blob Inventory rule name
container = "demodata" #Scoped container for the Blob Inventory
output_type = "csv" #Blob Inventory Output Type

storage_prefix = "/mnt/demodata/" #Mount to storage container for delta tables
lakehouse = "Delta_Inventory" #Lakehouse or Schema to store the inventory output in
inventory_path = "/mnt/demodata/" #Mount to storage container for blob inventory files
parallelism = 5
track_history = False #Use if you want to analyze delta inventory over time

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {lakehouse}")

In [0]:
delta_inventory = DeltaStorageInventory(session= spark, \
    storage_prefix=storage_prefix, 
    container=container,
    target_lakehouse=lakehouse, 
    parallelism=parallelism,
    track_history=track_history)

delta_inventory.run_from_storage_inventory(rule=inventory_rule, \
    inventory_data_path=inventory_path, \
    inventory_output_type=output_type)

# Example Delta Inventory Queries

In [0]:
%sql
SELECT
  delta_table,
  array_min(delta_versions) as min_version,
  array_max(delta_versions) as max_version
FROM delta_inventory.delta_tables

In [0]:
%sql
SELECT *
FROM delta_inventory.delta_table_snapshot

In [0]:
%sql
SELECT
  delta_table,
  count(DISTINCT delta_partition) as partition_count,
  sum(files_count) as file_count,
  sum(file_size) as file_size,
  sum(removed_files_count) as removed_files_count,
  sum(removed_file_size) as removed_file_size
FROM delta_inventory.delta_table_partitions
GROUP BY delta_table

In [0]:
%sql
SELECT
  DISTINCT operation
FROM delta_inventory.delta_table_history

In [0]:
%sql
SELECT * FROM (
  SELECT
    delta_table,
    operation
  FROM delta_inventory.delta_table_history
  WHERE operation NOT IN (
    'CREATE OR REPLACE TABLE AS SELECT',
    'CREATE TABLE AS SELECT')
)
PIVOT (
  COUNT(*)
  FOR operation in (
    'MERGE',
    'WRITE',
    'VACUUM END',
    'VACUUM START',
    'OPTIMIZE'
  )
)

In [0]:
%sql
SELECT
  f.delta_table,
  f.file_info["delta_version"] as delta_version,
  h.`timestamp` as version_dt,
  SUM(CASE WHEN f.file_info["operation"] = "ADD" THEN f.file_info["file_size"] ELSE 0 END) as active_size,
  SUM(CASE WHEN f.file_info["operation"] = "REMOVE" THEN f.file_info["file_size"] ELSE 0 END) as out_of_scope_size
FROM delta_inventory.delta_table_files f
JOIN delta_inventory.delta_table_history h ON f.delta_table_id=h.delta_table_id 
  AND f.file_info["delta_version"] = h.version
  AND f.inventory_date=h.inventory_date
GROUP BY
  f.delta_table,
  delta_version,
  version_dt